In [1]:
import numpy as np
import pickle
import os

seed = 2023

In [2]:
from nlpsig_networks.scripts.swmhau_network_functions import (
    swmhau_network_hyperparameter_search
)

In [3]:
output_dir = "client_talk_type_output"
if not os.path.isdir(output_dir):
    os.makedirs(output_dir)

## AnnoMI

In [4]:
%run ../load_anno_mi.py

In [5]:
anno_mi.head()

,mi_quality,transcript_id,topic,utterance_id,interlocutor,timestamp,utterance_text,annotator_id,therapist_input_exists,therapist_input_subtype,reflection_exists,reflection_subtype,question_exists,question_subtype,main_therapist_behaviour,client_talk_type,datetime
0,high,0,reducing alcohol consumption,0,therapist,00:00:13,Thanks for filling it out. We give this form t...,3,False,NaN,False,NaN,True,open,question,NaN,2023-08-17 00:00:13
1,high,0,reducing alcohol consumption,1,client,00:00:24,Sure.,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,neutral,2023-08-17 00:00:24
2,high,0,reducing alcohol consumption,2,therapist,00:00:25,"So, let's see. It looks that you put-- You dri...",3,True,information,False,NaN,False,NaN,therapist_input,NaN,2023-08-17 00:00:25
3,high,0,reducing alcohol consumption,3,client,00:00:34,Mm-hmm.,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,neutral,2023-08-17 00:00:34
4,high,0,reducing alcohol consumption,4,therapist,00:00:34,-and you usually have three to four drinks whe...,3,True,information,False,NaN,False,NaN,therapist_input,NaN,2023-08-17 00:00:34


In [6]:
with open("../anno_mi_sbert.pkl", "rb") as f:
    sbert_embeddings = pickle.load(f)
    
sbert_embeddings.shape

(9699, 384)

# swmhau Network

## Obtaining path by looking at post history

We can obtain a path by looking at the history of each post. Here we look at the last 10 posts (and pad with vectors of zeros if there are less than 10 posts) including the current post.

We only want to consider paths that correspond to a client's utterance as we want to model a change in mood at that time. Their history will still contain the therapist's utterances too.

In [7]:
time_features = ["time_encoding", "timeline_index"]
standardise_method = ["minmax", None]
num_features = len(time_features)
add_time_in_path = True

In [8]:
num_epochs = 100
embedding_dim = 384
dimensions = [15] # [50, 15]
# define swmhau parameters: (output_channels, sig_depth, num_heads)
swmhau_parameters = [(12, 3, 10), (8, 4, 6), (8, 4, 12)]
num_layers = [1]
ffn_hidden_dim_sizes = [[64,64],[256,256],[512,512]]
dropout_rates = [0.5, 0.1]
learning_rates = [1e-3, 1e-4, 5e-4]
seeds = [1]
loss = "focal"
gamma = 2
validation_metric = "f1"
patience = 5

In [9]:
# size = 20
# swmhau_network_umap_kfold_20, best_swmhau_network_umap_kfold_20, _, __ = swmhau_network_hyperparameter_search(
#     num_epochs=num_epochs,
#     df=anno_mi,
#     id_column="transcript_id",
#     label_column="client_talk_type",
#     embeddings=sbert_embeddings,
#     y_data=y_data_client,
#     embedding_dim=embedding_dim,
#     output_dim=output_dim_client,
#     history_lengths=[size],
#     dim_reduce_methods=["umap"],
#     dimensions=dimensions,
#     log_signature=True,
#     swmhau_parameters=swmhau_parameters,
#     num_layers=num_layers,
#     ffn_hidden_dim_sizes=ffn_hidden_dim_sizes,
#     dropout_rates=dropout_rates,
#     learning_rates=learning_rates,
#     seeds=seeds,
#     loss=loss,
#     gamma=gamma,
#     features=time_features,
#     standardise_method=standardise_method,
#     add_time_in_path=add_time_in_path,
#     path_indices=client_index,
#     k_fold=True,
#     patience=patience,
#     validation_metric=validation_metric,
#     results_output=f"{output_dir}/swmhau_network_umap_focal_{gamma}_{size}_kfold.csv",
#     verbose=False
# )

In [10]:
size = 20
swmhau_network_grp_kfold_20, best_swmhau_network_grp_kfold_20, _, __ = swmhau_network_hyperparameter_search(
    num_epochs=num_epochs,
    df=anno_mi,
    id_column="transcript_id",
    label_column="client_talk_type",
    embeddings=sbert_embeddings,
    y_data=y_data_client,
    embedding_dim=embedding_dim,
    output_dim=output_dim_client,
    history_lengths=[size],
    dim_reduce_methods=["gaussian_random_projection"],
    dimensions=dimensions,
    log_signature=True,
    swmhau_parameters=swmhau_parameters,
    num_layers=num_layers,
    ffn_hidden_dim_sizes=ffn_hidden_dim_sizes,
    dropout_rates=dropout_rates,
    learning_rates=learning_rates,
    seeds=seeds,
    loss=loss,
    gamma=gamma,
    features=time_features,
    standardise_method=standardise_method,
    path_indices=client_index,
    k_fold=True,
    patience=patience,
    validation_metric=validation_metric,
    results_output=f"{output_dir}/swmhau_network_grp_focal_{gamma}_{size}_kfold.csv",
    verbose=False
)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]


##################################################
dimension: 15 | method: gaussian_random_projection
[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/9699 [00:00<?, ?it/s]

...

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
swmhau_network_grp_kfold_20.groupby(["dimensions",
                                     "output_channels",
                                     "sig_depth",
                                     "num_heads",
                                     "num_layers",
                                     "ffn_hidden_dim",
                                     "learning_rate"]).mean()

/var/folders/l7/bnn1j6bs3bqfskq2jlq55yh80000gr/T/ipykernel_7649/809976104.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  swmhau_network_grp_kfold_20.groupby(["dimensions",


accuracy  \
dimensions output_channels sig_depth num_heads num_layers ffn_hidden_dim learning_rate             
15         12              3         10        1          (64, 64)       0.0001         0.673552   
                                                                         0.0005         0.674071   
                                                                         0.0010         0.674901   
                                                          (256, 256)     0.0001         0.661408   
                                                                         0.0005         0.673241   
                                                                         0.0010         0.671995   
                                                          (512, 512)     0.0001         0.665456   
                                                                         0.0005         0.663484   
                                                                         0.0010         0.667843   

                                                                                              f1  \
dimensions output_channels sig_depth num_heads num_layers ffn_hidden_dim learning_rate             
15         12              3         10        1          (64, 64)       0.0001         0.552367   
                                                                         0.0005         0.553772   
                                                                         0.0010         0.549297   
                                                          (256, 256)     0.0001         0.535620   
                                                                         0.0005         0.552635   
                                                                         0.0010         0.545499   
                                                          (512, 512)     0.0001         0.532737   
                                                                         0.0005         0.547324   
                                                                         0.0010         0.533039   

                                                                                        precision  \
dimensions output_channels sig_depth num_heads num_layers ffn_hidden_dim learning_rate              
15         12              3         10        1          (64, 64)       0.0001          0.552802   
                                                                         0.0005          0.554085   
                                                                         0.0010          0.556696   
                                                          (256, 256)     0.0001          0.539904   
                                                                         0.0005          0.555711   
                                                                         0.0010          0.553019   
                                                          (512, 512)     0.0001          0.537866   
                                                                         0.0005          0.548569   
                                                                         0.0010          0.544581   

                                                                                          recall  \
dimensions output_channels sig_depth num_heads num_layers ffn_hidden_dim learning_rate             
15         12              3         10        1          (64, 64)       0.0001         0.554272   
                                                                         0.0005         0.554241   
                                                                         0.0010         0.546562   
                                                          (256, 256)     0.0001         0.537865   
                                                                         0.0005         0.553691   
                                                                         0

In [ ]:
best_swmhau_network_grp_kfold_20

,loss,accuracy,f1,f1_scores,precision,precision_scores,recall,recall_scores,valid_loss,valid_accuracy,...,learning_rate,seed,loss_function,gamma,k_fold,n_splits,augmentation_type,hidden_dim_aug,comb_method,batch_size
0,None,0.674486,0.55747,"[0.7887233701745956, 0.5045672031317964, 0.379...",0.558348,"[0.7833280305440662, 0.5173951828724354, 0.374...",0.556866,"[0.7941935483870968, 0.4923599320882852, 0.384...",None,0.670912,...,0.001,1,focal,2,True,5,Conv1d,None,concatenation,64


In [ ]:
best_swmhau_network_grp_kfold_20["f1"].mean()

0.5574704841424237

In [ ]:
best_swmhau_network_grp_kfold_20["precision"].mean()

0.5583483646891653

In [ ]:
best_swmhau_network_grp_kfold_20["recall"].mean()

0.5568660024590172

In [ ]:
np.stack(best_swmhau_network_grp_kfold_20["f1_scores"]).mean(axis=0)

array([0.78872337, 0.5045672 , 0.37912088])

In [ ]:
np.stack(best_swmhau_network_grp_kfold_20["precision_scores"]).mean(axis=0)

array([0.78332803, 0.51739518, 0.37432188])

In [ ]:
np.stack(best_swmhau_network_grp_kfold_20["recall_scores"]).mean(axis=0)

array([0.79419355, 0.49235993, 0.38404453])